In [1]:
import pandas as pd
import numpy as np
import os
import csv
import gzip
import sys
import datetime
import ast
import matplotlib.pyplot as plt
from collections import Counter
import geopandas as gpd

The mapping of the data is as follows:

- ZCTA to MODZCTA, this is created by the NYC DOHMH
- zip to zcta is created by a lab by identifying the boundaries of the ZCTA and ZIP codes
- Thus, we have zip code to MODZCTA mapping (outer merge) 
several zipcodes belong to the same MODZCTA
- Then, NYC DOHMH has a list of zipcodes they care, we filter out the zip codes that are not in the list
- Tract to zipcode has covered the whole usa, we could add tract to the mapping
- TRACT to zip is not unique, we use the resident ratio to assign the tract to the zip code, [(tract1, V1)...]-->[(zip1, V1*ratio_(tract1inzip2)), (zip2, V1*ratio_(tract1inzip2))...]

**Use the zipcode to modzcta mapping to get the modzcta for each zipcode**

**Use the modzcta to tract mapping to get the tract for each modzcta** 

**They can not be used interchangeably**.

In [2]:
zcta_modzcta = pd.read_csv('../data/mappings/ZCTA-to-MODZCTA.csv') 
##nyc health data, however, 
## not all the zcta and modzcta belongs to nyc: 11001 , 11003, 11040,(Nassian county, long island)
#  11371( La Guardia Airport), 11430 (JFK Airport) 99999(Central park, unkown)
zcta_modzcta.head()## int
zcta_modzcta['ZCTA_str'] = zcta_modzcta.ZCTA.astype(str)
print(zcta_modzcta['ZCTA'].nunique())
print(zcta_modzcta['MODZCTA'].nunique())

215
178


In [3]:
df_zip_zcta = pd.read_csv('../data/mappings/zip_zcta_xref.csv') ### this is acurate actully
#https://github.com/censusreporter/acs-aggregate/tree/master
modzcta_zip = zcta_modzcta.merge(df_zip_zcta, left_on='ZCTA', right_on='zcta', how='left')
zipcode_counts = pd.read_csv('../results/counts/zipcode_counts.csv')
############# check the number of zipcodes in the survey data
### there are some zipcodes are not considered by the nyc government ###
# survey_zipcode_ls = zipcode_counts[zipcode_counts['Count']>10]['zip_code'].values.tolist()
# df_selected = modzcta_zip[modzcta_zip['zip_code'].isin(survey_zipcode_ls)]
# check_ls = df_selected['MODZCTA'].values.tolist()
# len(modzcta_zip[modzcta_zip['MODZCTA'].isin(check_ls)]) ### not equal to the df selected 
modzcta_zip_counts = pd.merge(modzcta_zip, zipcode_counts, left_on='zip_code', right_on='zip_code', how='outer')

In [4]:
df_tai = pd.read_excel('../Data/mappings/2_uhfOrder.xlsx') 
### just used to order the zipcodes
### this zip is not ZCTA ! i have amnually checked the zipcodes
df_tai['zip'] = df_tai['Zipcodes'].str.split(',')
combined_list = []
for sublist in df_tai['zip']:
    combined_list.extend(sublist)
nyc_zip_ls = [int(i) for i in combined_list] #### the order is not be changed
zip_order = pd.DataFrame({'zip_nyc':nyc_zip_ls}) 
zip_order['zip_idx'] = zip_order['zip_nyc'].index
print(len(nyc_zip_ls))

209


In [5]:
set(zcta_modzcta['ZCTA'].unique()) - set(nyc_zip_ls)

{11001, 11003, 11040, 11371, 11430, 99999}

In [6]:
modzcta_zip_counts = modzcta_zip_counts[modzcta_zip_counts['zip_code'].isin(nyc_zip_ls)]
modzcta_zip_counts = modzcta_zip_counts.merge(zip_order, left_on='zip_code', right_on='zip_nyc', how='left')
modzcta_zip_counts.sort_values(by='zip_idx',ascending=True,inplace=True)

In [7]:
ny_modzcat_ls = modzcta_zip_counts.drop_duplicates(subset='MODZCTA', keep='first')['MODZCTA'].to_list()
ny_modzcat_order = pd.DataFrame({'MODZCTA':ny_modzcat_ls})
ny_modzcat_order['mod_idx'] = ny_modzcat_order['MODZCTA'].index

In [8]:
# modzcta_zip_counts
grouped_modzcta = modzcta_zip_counts.groupby('MODZCTA')['Count'].sum().reset_index().rename(columns={'Count': 'modzcta_count'})
modzcta_zip_counts = modzcta_zip_counts.merge(grouped_modzcta, on='MODZCTA', how='left')
sorted_modzcta_counts = modzcta_zip_counts.sort_values(by='modzcta_count',ascending=False)
sorted_modzcta_counts = sorted_modzcta_counts.drop(columns=['zcta','source'])
df_modzcta = sorted_modzcta_counts.merge(ny_modzcat_order, on='MODZCTA', how='left').drop(columns=['zip_nyc','zip_idx'])
df_modzcta.to_csv('../results/counts/modzcta_zip_counts.csv', index=False)
### the zipcode are individually mapped to the MODZCTA, except for 
#10005	10005	10005	 Lower Manhattan (Manhattan) 18
#10005	10271	10271	 Lower Manhattan (Manhattan) 1
######################################################
#10007	10007	10007	Lower Manhattan (Manhattan)	12 
#10007	10279	10279	Lower Manhattan (Manhattan)	1	

In [9]:
len(df_modzcta) ### should the number of the zipcodes in the nyc

209

In [10]:
portions = pd.read_excel('../data/mappings/TRACT_ZIP_122021.xlsx')
# portions = pd.read_excel('../data/TRACT_ZIP_032020.xlsx')
### it covers the whole usa 

In [11]:
### merge the with the tract data 
df_tract_modzcta = pd.merge(df_modzcta, portions, left_on='zip_code', right_on='zip', how='outer') #2021
# df_tract_modzcta = pd.merge(df_modzcta, portions, left_on='zip_code', right_on='ZIP', how='outer') #2020
nyc_tract_modzcta = df_tract_modzcta[df_tract_modzcta['zip_code'].isin(nyc_zip_ls)].reset_index(drop=True) ### there are some zipcodes are not in the tract data

In [12]:
### add borough information
df_borough = pd.read_csv('../Data/mappings/data-by-modzcta.csv')
df_borough['borough'] = df_borough['BOROUGH_GROUP'].replace({'Manhattan':'MN', 'Brooklyn':'BK', 'Queens':'QN', 'Bronx':'BX', 'Staten Island':'SI'})
df_borough_s = df_borough[['MODIFIED_ZCTA','borough']].copy().rename(columns={'MODIFIED_ZCTA':'MODZCTA'})
nyc_tract_modzcta = nyc_tract_modzcta.merge(df_borough_s, on='MODZCTA', how='left')

In [13]:
nyc_tract_modzcta.to_csv('../data/mappings/nyc_tract_modzcta_v2021.csv', index=False)

In [14]:
nyc_tract_modzcta['mod_idx'].nunique() 

177

#### the following is the version to do the one to one mapping, it will reduce the visits of some zipcodes, like 10001, it should be used in the POI mapping

In [15]:
filtered_df = nyc_tract_modzcta.groupby('tract').filter(lambda x: len(x) > 1)
# filtered_df.to_csv('../results/filtered_df.csv', index=False)

In [16]:
GOEID_MULTIPLES = filtered_df['tract'].unique()
print(len(GOEID_MULTIPLES))

805


In [17]:
### one GEOID has multiple MODZCTA, we choose the one with the most RES
### one zip code has multiple MODZCTA, 
for id_tochek in GOEID_MULTIPLES:
    subdf = nyc_tract_modzcta[nyc_tract_modzcta['tract']==id_tochek]
    max_idx = subdf['tot_ratio'].idxmax()
    # Retrieve the row with the maximum value in 'col1'
    max_row = subdf.loc[max_idx]
    zip_to_use = max_row['zip'] ### int 
    # nyc_tract_modzcta = nyc_tract_modzcta[~((nyc_tract_modzcta['TRACT']==id_tochek) & (nyc_tract_modzcta['ZIP']!=zip_to_use))]
    sub_idx_ls = subdf.index.to_list()
    sub_idx_ls.remove(max_idx)
    nyc_tract_modzcta.drop(sub_idx_ls, inplace=True)

In [18]:
nyc_tract_modzcta.to_csv('../data/mappings/nyc_tract_modzcta_poi_v2021.csv', index=False)

In [19]:
nyc_tract_modzcta.groupby('tract').filter(lambda x: len(x) > 1)

,ZCTA,MODZCTA,ZCTA_str,zip_code,Value,Count,modzcta_count,mod_idx,tract,zip,usps_zip_pref_city,usps_zip_pref_state,res_ratio,bus_ratio,oth_ratio,tot_ratio,borough


In [20]:
nyc_tract_modzcta['tract'].nunique() == len(nyc_tract_modzcta)

True